In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functions_NLM import estimate_nested_logit, simulate_choice
from functions_geom_DIB import geom_DIB_on_alternatives

In [9]:
# Load data
data = pd.read_csv('../data/airline.dat', sep='\t')
# display every column of the dataframe
pd.set_option('display.max_columns', None)
data

,SubjectId,OriginGMT,DestinationGMT,Direction,q02_TripPurpose,q03_WhoPays,q11_DepartureOrArrivalIsImportant,q12_IdealDepTime,q13_IdealArrTime,q14_PartySize,q15_Age,q16_Income,Cont_Income,q17_Gender,q19_Occupation,q20_Education,AirlineFirstFlight_1,AirlineFirstFlight_2,AirlineFirstFlight_3,AirlineSecondFlight_1,AirlineSecondFlight_2,AirlineSecondFlight_3,AirplaneFirstFlight_1,AirplaneFirstFlight_2,AirplaneFirstFlight_3,AirplaneSecondFlight_1,AirplaneSecondFlight_2,AirplaneSecondFlight_3,DepartureTimeHours_1,DepartureTimeHours_2,DepartureTimeHours_3,DepartureTimeMins_1,DepartureTimeMins_2,DepartureTimeMins_3,ArrivalTimeHours_1,ArrivalTimeHours_2,ArrivalTimeHours_3,ArrivalTimeMins_1,ArrivalTimeMins_2,ArrivalTimeMins_3,FlyingTimeHours_1,FlyingTimeHours_2,FlyingTimeHours_3,TripTimeHours_1,TripTimeHours_2,TripTimeHours_3,Legroom_1,Legroom_2,Legroom_3,Fare_1,Fare_2,Fare_3,BestAlternative_1,BestAlternative_2,BestAlternative_3,TripPurpose
0,1,300,360,0,2,1,2,-1,480,2,3.5,9,92.5,1,2,6,5,6,3,-1,-1,2,2,3,7,-1,8,7,7.0,9.0,9.0,420,540,540,10.6167,14.1167,13.6167,637,847,817,4.616670,5.11667,5.11667,4.616670,6.11667,5.61667,2,3,4,835,835,730,0,0,1,2
1,2,480,480,1,2,1,2,-1,540,1,3.5,10,112.5,1,1,6,3,4,5,-1,-1,1,8,8,2,-1,8,1,7.5,15.0,12.0,450,900,720,10.0333,18.5333,16.5333,602,1112,992,2.533330,3.03333,3.03333,2.533330,3.53333,4.53333,2,2,3,315,315,350,0,1,0,2
2,3,480,480,1,2,1,1,-1,-1,2,3.5,6,60.0,1,2,6,5,9,2,-1,-1,6,8,5,5,-1,6,6,7.5,9.0,12.0,450,540,720,10.0333,13.5333,17.0333,602,812,1022,2.533330,3.03333,3.03333,2.533330,4.53333,5.03333,3,1,4,195,160,170,0,1,0,2
3,4,480,480,1,2,3,1,630,-1,2,6.0,12,162.5,2,2,6,4,3,7,-1,-1,6,7,4,6,-1,6,8,18.0,18.0,12.0,1080,1080,720,18.9667,21.4667,13.9667,1138,1288,838,0.966667,1.46667,1.46667,0.966667,3.46667,1.96667,3,3,1,135,125,140,0,0,1,2
4,5,300,480,0,2,1,0,-1,-1,3,5.0,10,112.5,2,5,7,7,1,4,-1,-1,3,3,2,2,-1,3,6,10.5,15.0,15.0,630,900,900,11.8000,18.3000,17.8000,708,1098,1068,4.300000,4.80000,4.80000,4.300000,6.30000,5.80000,3,2,3,270,300,300,1,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3604,3609,480,300,2,3,1,2,-1,660,3,4.0,3,30.0,2,5,2,7,3,3,-1,-1,9,5,7,5,-1,2,3,12.0,11.0,9.0,720,660,540,19.2667,19.2667,18.2667,1156,1156,1096,4.266670,4.76667,4.76667,4.266670,5.26667,6.26667,3,3,4,650,480,650,0,1,0,3
3605,3610,480,300,2,2,1,2,-1,480,1,3.0,5,50.0,1,2,7,1,2,7,-1,-1,10,7,8,8,-1,6,4,12.0,10.0,7.0,720,600,420,20.3333,19.8333,16.3333,1220,1190,980,5.333330,5.83333,5.83333,5.333330,6.83333,6.33333,2,3,4,760,690,590,0,0,1,2
3606,3611,300,360,0,1,1,2,-1,870,1,4.0,8,80.0,1,7,7,10,1,5,-1,-1,3,4,4,3,-1,3,6,9.0,16.5,12.0,540,990,720,10.3000,18.8000,14.3000,618,1128,858,2.300000,2.80000,2.80000,2.300000,3.30000,3.30000,2,1,4,290,290,300,0,0,1,1
3607,3612,300,480,0,2,1,2,-1,1320,1,3.0,10,112.5,1,1,7,7,5,4,-1,-1,6,5,1,6,-1,5,5,10.0,11.0,11.0,600,660,660,13.1500,16.6500,15.6500,789,999,939,6.150000,6.65000,6.65000,6.150000,8.65000,7.65000,3,4,3,480,360,420,1,0,0,2


In [10]:
# Find the column index with the maximum value (1)
choice_column_index = data[['BestAlternative_1', 'BestAlternative_2', 'BestAlternative_3']].idxmax(axis=1)

# Add 1 to the index to get the choice number
data['Choice'] = choice_column_index.str[-1].astype(int)

data_used = data[['SubjectId', 'TripTimeHours_1', 'TripTimeHours_2', 'TripTimeHours_3',
                   'Fare_1', 'Fare_2', 'Fare_3', 'Choice']]
data_used

,SubjectId,TripTimeHours_1,TripTimeHours_2,TripTimeHours_3,Fare_1,Fare_2,Fare_3,Choice
0,1,4.616670,6.11667,5.61667,835,835,730,3
1,2,2.533330,3.53333,4.53333,315,315,350,2
2,3,2.533330,4.53333,5.03333,195,160,170,2
3,4,0.966667,3.46667,1.96667,135,125,140,3
4,5,4.300000,6.30000,5.80000,270,300,300,1
...,...,...,...,...,...,...,...,...
3604,3609,4.266670,5.26667,6.26667,650,480,650,2
3605,3610,5.333330,6.83333,6.33333,760,690,590,3
3606,3611,2.300000,3.30000,3.30000,290,290,300,3
3607,3612,6.150000,8.65000,7.65000,480,360,420,1


In [11]:
# beta_names = ["ASC_2", "ASC_3", "BETA_Time", "BETA_Cost", "lambda_21",
#                "lambda_13", "base_alpha_1_WITH_2"]

# DEFINE MODEL STRUCTURE AND LIKELIHOOD FUNCTION
def log_likelihood_optima_CNLM(beta, data):
    alpha_1_WITH_2 = np.exp(beta[6]) / (1 + np.exp(beta[6]))
    alpha_1_WITH_3 = 1 - alpha_1_WITH_2

    # define utility functions
    data['U_1'] = beta[2] * data['TripTimeHours_1'] + beta[3] * data['Fare_1']
    data['U_2'] = beta[0] + beta[2] * data['TripTimeHours_2'] + beta[3] * data['Fare_2']
    data['U_3'] = beta[1] + beta[2] * data['TripTimeHours_3'] + beta[3] * data['Fare_3']
 
    # combined utility terms inside nests
    data['U_21'] = (alpha_1_WITH_2 * np.exp(data['U_1'])) ** (1 / beta[4]) + \
                          (1 * np.exp(data['U_2'])) ** (1 / beta[4])
    data['U_13'] = (alpha_1_WITH_3 * np.exp(data['U_1'])) ** (1 / beta[5]) + \
                         (1 * np.exp(data['U_3'])) ** (1 / beta[5])

    # Nest probabilities
    data['P_nest_21'] = data['U_21'] ** beta[4] / \
                                (data['U_21'] ** beta[4] + data['U_13'] ** beta[5])
    data['P_nest_13'] = 1 - data['P_nest_21']

    # Within nest probabilities for any nests with > 1 alt
    data['P_1_in_21'] = (alpha_1_WITH_2 * np.exp(data['U_1'])) ** (1 / beta[4]) / \
                                    data['U_21'] 
    data['P_2_in_21'] = 1 - data['P_1_in_21']
    data['P_1_in_13'] = (alpha_1_WITH_3 * np.exp(data['U_1'])) ** (1 / beta[5]) / \
                                data['U_13']
    data['P_3_in_13'] = 1 - data['P_1_in_13']

    # Full probabilities
    data['P_1'] = data['P_nest_21'] * data['P_1_in_21'] + \
                            data['P_nest_13'] * data['P_1_in_13']   
    data['P_2'] = data['P_nest_21'] * data['P_2_in_21']
    data['P_3'] = data['P_nest_13'] * data['P_3_in_13']

    # calculate probability for chosen alternative for each row in the data table
    data['P'] = ((data['Choice'] == 1) * data['P_1'] +
                 (data['Choice'] == 2) * data['P_2'] +
                 (data['Choice'] == 3) * data['P_3'])

    # take product across choices for the same person (likelihood)
    # then take the log for log-likelihood
    LL = data.groupby('SubjectId')['P'].prod().apply(np.log).sum()

    return -LL  # We minimize negative log-likelihood

In [12]:
# Define model parameters
beta = np.array([0, 0, 0, 0, 1, 1, 1])
beta_names = ["ASC_2", "ASC_3", "BETA_Time", "BETA_Cost", "lambda_21",
              "lambda_13", "base_alpha_1_WITH_2"]
# Estimate parameters
result, se, t_stat, p_value, aic, bic  = estimate_nested_logit(data_used, beta, beta_names, log_likelihood_optima_CNLM)

/var/folders/gk/16047hxn3hdgt8s5qmq_gnqh0000gp/T/ipykernel_14102/2971109677.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['U_1'] = beta[2] * data['TripTimeHours_1'] + beta[3] * data['Fare_1']
/var/folders/gk/16047hxn3hdgt8s5qmq_gnqh0000gp/T/ipykernel_14102/2971109677.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['U_2'] = beta[0] + beta[2] * data['TripTimeHours_2'] + beta[3] * data['Fare_2']
/var/folders/gk/16047hxn3hdgt8s5qmq_gnqh0000gp/T/ipykernel_14102/2971109677.py:12: SettingWith

Optimization Results:
             Parameter  Estimate  Robust Asymptotic SE  t-statistic  \
0                ASC_2 -1.119182              0.235042    -4.761626   
1                ASC_3 -1.336384              0.140122    -9.537295   
2            BETA_Time -0.254901              0.038340    -6.648441   
3            BETA_Cost -0.015433              0.003199    -4.824539   
4            lambda_21  0.853476              0.194329     4.391918   
5            lambda_13  0.511141              0.162054     3.154132   
6  base_alpha_1_WITH_2  1.722964              0.621786     2.770993   

        p-value  
0  1.996003e-06  
1  0.000000e+00  
2  3.409117e-11  
3  1.461437e-06  
4  1.155866e-05  
5  1.623013e-03  
6  5.617241e-03  
AIC: 4858.311730592237
BIC: 4901.650032624736
Log-likelihood: -2422.1558652961185


/var/folders/gk/16047hxn3hdgt8s5qmq_gnqh0000gp/T/ipykernel_14102/2971109677.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['U_1'] = beta[2] * data['TripTimeHours_1'] + beta[3] * data['Fare_1']
/var/folders/gk/16047hxn3hdgt8s5qmq_gnqh0000gp/T/ipykernel_14102/2971109677.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['U_2'] = beta[0] + beta[2] * data['TripTimeHours_2'] + beta[3] * data['Fare_2']
/var/folders/gk/16047hxn3hdgt8s5qmq_gnqh0000gp/T/ipykernel_14102/2971109677.py:12: SettingWith

In [13]:
df_input = data[['TripTimeHours_1', 'TripTimeHours_2', 'TripTimeHours_3',
                   'Fare_1', 'Fare_2', 'Fare_3']]
# Calculate frequencies and probabilities
vector_counts = df_input.value_counts().reset_index(name='Frequency')
vector_counts['Probability'] = vector_counts['Frequency'] / len(df_input)

# Creating a tuple of attributes to facilitate mapping
vector_counts['tuple'] = vector_counts[['TripTimeHours_1', 'TripTimeHours_2', 'TripTimeHours_3',
                                        'Fare_1', 'Fare_2', 'Fare_3']].apply(tuple, axis=1)
probability_map = vector_counts.set_index('tuple')['Probability'].to_dict()

# Assign the probability to each row based on its tuple of attributes
df_input['Probability'] = df_input.apply(lambda row: probability_map[tuple(row)], axis=1)


/var/folders/gk/16047hxn3hdgt8s5qmq_gnqh0000gp/T/ipykernel_14102/1916923433.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input['Probability'] = df_input.apply(lambda row: probability_map[tuple(row)], axis=1)


In [14]:
# Computation of p(x,y)
p_x = df_input['Probability'].values

p_y_given_x = data_used[['P_1', 'P_2', 'P_3']].values
p_xy = p_x[:, np.newaxis] * p_y_given_x

# Normalize p_xy 
p_xy /= p_xy.sum()

# Define epsilon value
epsilon = 1e-20

# Add epsilon to elements equal to 0
p_xy[p_xy == 0] += epsilon

In [15]:
def simulate_choice_optima(row):
    """
    Function to simulate a choice based on cumulative probabilities.

    Parameters:
    - row: pandas Series or DataFrame row containing probabilities for each choice

    Returns:
    - The simulated choice (an integer between 1 and the number of choices)
    """
    # Generate a random number between 0 and 1
    random_number = np.random.rand()
    
    # Calculate cumulative probabilities
    probabilities = row[['P_1', 'P_2', 'P_3']]
    cumulative_probs = probabilities.cumsum()
    
    # Determine simulated choice based on random number
    for i, cumulative_prob in enumerate(cumulative_probs):
        if random_number <= cumulative_prob:
            return i+1

In [16]:
np.random.seed(123)
data_used['simulated_choice'] = data_used.apply(simulate_choice_optima, axis=1)
data_used

/var/folders/gk/16047hxn3hdgt8s5qmq_gnqh0000gp/T/ipykernel_14102/2856924636.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_used['simulated_choice'] = data_used.apply(simulate_choice_optima, axis=1)


,SubjectId,TripTimeHours_1,TripTimeHours_2,TripTimeHours_3,Fare_1,Fare_2,Fare_3,Choice,U_1,U_2,U_3,U_21,U_13,P_nest_21,P_nest_13,P_1_in_21,P_2_in_21,P_1_in_13,P_3_in_13,P_1,P_2,P_3,P,simulated_choice
0,1,4.616670,6.11667,5.61667,835,835,730,3,-14.063613,-15.565146,-14.034400,6.956684e-08,1.218015e-12,0.489101,0.510899,0.827333,0.172667,0.022992,0.977008,0.416396,0.084452,0.499152,0.499152,3
1,2,2.533330,3.53333,4.53333,315,315,350,2,-5.507243,-6.881325,-7.893595,1.615319e-03,7.178543e-07,0.851291,0.148709,0.804948,0.195052,0.726409,0.273591,0.793268,0.166047,0.040685,0.166047,1
2,3,2.533330,4.53333,5.03333,195,160,170,2,-3.655245,-4.744062,-5.243048,1.524182e-02,5.462487e-05,0.809406,0.190594,0.747112,0.252888,0.357596,0.642404,0.672873,0.204689,0.122438,0.204689,1
3,4,0.966667,3.46667,1.96667,135,125,140,3,-2.329902,-3.932003,-3.998354,6.378698e-02,6.617882e-04,0.801054,0.198946,0.843522,0.156478,0.394597,0.605403,0.754210,0.125347,0.120443,0.120443,1
4,5,4.300000,6.30000,5.80000,270,300,300,1,-5.263070,-7.355053,-7.444805,1.911774e-03,1.313338e-06,0.829178,0.170822,0.905395,0.094605,0.640184,0.359816,0.860091,0.078445,0.061464,0.860091,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3604,3609,4.266670,5.26667,6.26667,650,480,650,2,-11.119234,-9.869653,-12.965420,1.131416e-05,1.852506e-11,0.948369,0.051631,0.160221,0.839779,0.479933,0.520067,0.176728,0.796420,0.026852,0.796420,2
3605,3610,5.333330,6.83333,6.33333,760,690,590,3,-13.088792,-13.509992,-12.056413,3.138240e-07,5.722662e-11,0.326164,0.673836,0.574696,0.425304,0.003295,0.996705,0.189665,0.138719,0.671616,0.671616,3
3606,3611,2.300000,3.30000,3.30000,290,290,300,3,-5.061934,-6.436016,-6.807552,2.721796e-03,2.890182e-06,0.814294,0.185706,0.804948,0.195052,0.431174,0.568826,0.735535,0.158830,0.105635,0.105635,1
3607,3612,6.150000,8.65000,7.65000,480,360,420,1,-8.975633,-8.880069,-9.768369,5.263488e-05,5.603625e-09,0.786345,0.213655,0.424471,0.575529,0.105148,0.894852,0.356246,0.452564,0.191190,0.356246,2


In [17]:
betas = np.linspace(0, 50, 51)
# Initialize an empty list to store the number of clusters
num_clusters_list = []

# Iterate over each beta value
for beta in betas:
    # Run iterative_algorithm to obtain q_t_given_x
    q_t_given_x, _, _ = geom_DIB_on_alternatives(p_xy, max_iter=5000, beta=beta, threshold=1e-4)
    
    # Calculate the number of clusters
    column_sum = np.sum(q_t_given_x, axis=0)
    num_clusters = np.count_nonzero(column_sum)
    
    # Append the number of clusters to the list
    num_clusters_list.append(num_clusters)

# Plot the number of clusters against beta values
plt.plot(betas, num_clusters_list)
plt.xlabel('Beta')
plt.ylabel('Number of Clusters')
plt.title('Number of Clusters vs. Beta')
plt.grid(True)
plt.show()

Iteration: 1 out of 5000
Objective function value: 1.0985997641372804
H(T) =  1.0985997641372804
I(T;Y) =  0.0002956553858237676
Iteration: 2 out of 5000
Objective function value: 0.7958394297761275
H(T) =  0.7958394297761275
I(T;Y) =  6.5687594346997e-05
Iteration: 3 out of 5000
Objective function value: 0.5999047342041967
H(T) =  0.5999047342041967
I(T;Y) =  4.1982052875866316e-05
Iteration: 4 out of 5000
Objective function value: 0.48492981316986006
H(T) =  0.48492981316986006
I(T;Y) =  3.0848828054930166e-05
Iteration: 5 out of 5000
Objective function value: 0.40934537361324
H(T) =  0.40934537361324
I(T;Y) =  2.4382709879144926e-05
Iteration: 6 out of 5000
Objective function value: 0.35562971035672014
H(T) =  0.35562971035672014
I(T;Y) =  2.0157546209320643e-05
Iteration: 7 out of 5000
Objective function value: 0.3153324162088299
H(T) =  0.3153324162088299
I(T;Y) =  1.7180434024099966e-05
Iteration: 8 out of 5000
Objective function value: 0.2838855203136472
H(T) =  0.28388552031364

KeyboardInterrupt: 

In [21]:
q_t_given_x, q_t, q_y_given_t = geom_DIB_on_alternatives(p_xy, beta=10, max_iter=5000, threshold=1e-5)

Iteration: 1 out of 5000
Objective function value: 1.0956432102790428
H(T) =  1.0985997641372804
I(T;Y) =  0.0002956553858237676
Iteration: 2 out of 5000
Objective function value: 0.6867288880099138
H(T) =  1.0599110163629517
I(T;Y) =  0.03731821283530379
Iteration: 3 out of 5000
Objective function value: 0.06334681300369394
H(T) =  0.7352186529328685
I(T;Y) =  0.06718718399291745
Iteration: 4 out of 5000
Objective function value: -0.15458280268871594
H(T) =  0.6461163776487893
I(T;Y) =  0.08006991803375052
Iteration: 5 out of 5000
Objective function value: -0.18400400139103745
H(T) =  0.5884112416293393
I(T;Y) =  0.07724152430203768
Iteration: 6 out of 5000
Objective function value: -0.2000178726716325
H(T) =  0.5430001786199005
I(T;Y) =  0.0743018051291533
Iteration: 7 out of 5000
Objective function value: -0.21444422347609593
H(T) =  0.503847496070384
I(T;Y) =  0.071829171954648
Iteration: 8 out of 5000
Objective function value: -0.22345438402184803
H(T) =  0.4828953574584064
I(T;Y)

In [22]:
import re

# Define a function to extract the number from the column name
def extract_number(column_name):
    match = re.search(r'\d+', column_name)  # Search for one or more digits in the column name
    if match:
        return int(match.group())  # Return the first occurrence of digits as an integer
    else:
        return None  # Return None if no digits are found

In [23]:
# Calculate the number of clusters
column_sum = np.sum(q_t_given_x, axis=0)
num_clusters = np.count_nonzero(column_sum)
print('Number of clusters :', num_clusters)

# Count the number of observations in each cluster
data_used['cluster'] = np.argmax(q_t_given_x, axis=1)
print('\n', data_used['cluster'].value_counts())

# number of each alternative 1, 2, 3, 4, 5 in each cluster 
cluster_counts = data_used.groupby(['cluster', 'Choice']).size().unstack(fill_value=0)
print('\n', cluster_counts)

data_used['max_proba'] = data_used[['P_1', 'P_2', 'P_3']].idxmax(axis=1).apply(extract_number)
cluster_counts2 = data_used.groupby(['cluster', 'max_proba']).size().unstack(fill_value=0)
print('\n', cluster_counts2)

cluster_counts3 = data_used.groupby(['cluster', 'simulated_choice']).size().unstack(fill_value=0)
print('\n', cluster_counts3)

Number of clusters : 3

 cluster
1    3203
0     226
2     180
Name: count, dtype: int64

 Choice      1    2    3
cluster                
0          80  120   26
1        2351  459  393
2          73   10   97

 max_proba     1    2    3
cluster                  
0             0  184   42
1          3164   32    7
2             6    0  174

 simulated_choice     1    2    3
cluster                         
0                   60  121   45
1                 2379  459  365
2                   53   14  113


/var/folders/gk/16047hxn3hdgt8s5qmq_gnqh0000gp/T/ipykernel_14102/687862934.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_used['cluster'] = np.argmax(q_t_given_x, axis=1)
/var/folders/gk/16047hxn3hdgt8s5qmq_gnqh0000gp/T/ipykernel_14102/687862934.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_used['max_proba'] = data_used[['P_1', 'P_2', 'P_3']].idxmax(axis=1).apply(extract_number)
